# Opptaksområder 2016

OBS: lag fix for uoppgitte kommuner før den lastes opp!

In [1]:
aargang <- 2016

In [2]:
options(repr.matrix.max.rows=600, repr.matrix.max.cols=2000)

suppressPackageStartupMessages({ 
library(tidyverse)
library(readxl)
library(klassR)
library(sf)
library(leaflet)
        })

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”


### Filstier

In [3]:
arbeidsmappe <- "/ssb/stamme01/fylkhels/speshelse/felles/"
arbeidsmappe_kart <- paste0(arbeidsmappe, "kart/", aargang, "/")

arbeidsmappe_opptak <- paste0(arbeidsmappe, "opptaksomrader/", aargang, "/")

if (file.exists(arbeidsmappe_opptak)==FALSE) {
  dir.create(arbeidsmappe_opptak)
}

## Laster inn kart (grunnkrets)

In [4]:
# start.time <- Sys.time()

#   rename_geometry <- function(g, name){
#     current = attr(g, "sf_column")
#     names(g)[names(g)==current] = name
#     sf::st_geometry(g)=name
#     g
#   }

# # grunnkrets_kart_filsti <- paste0(arbeidsmappe_kart, "ABAS_grunnkrets_utenhav_", aargang, ".parquet")
# grunnkrets_kart_filsti <- paste0(arbeidsmappe_kart, "ABAS_grunnkrets_flate_", aargang, ".parquet")

# # Lese inn filen som parquet med sfarrow
# grunnkrets_kart <- sfarrow::st_read_parquet(grunnkrets_kart_filsti) %>%
# sf::st_zm(drop = T) %>%
# sf::st_cast("MULTIPOLYGON") %>%
#   sf::st_transform(crs = 4326) %>%
#   dplyr::rename(GRUNNKRETSNUMMER = GRUNNKRETS)

# grunnkrets_kart <- rename_geometry(grunnkrets_kart, "geometry")
# sf::st_geometry(grunnkrets_kart) <- "geometry"

# end.time <- Sys.time()
# time.taken <- end.time - start.time
# time.taken

In [5]:
# grunnkrets_kart_test <- grunnkrets_kart %>%
# dplyr::filter(GRUNNKRETSNUMMER %in% c("14440104", "14490113")) %>%
# sf::st_transform(crs = 4326)

In [6]:
# T04317 <- PxWebApiData::ApiData(04317, ContentsCode = "Personer", 
#                                 Grunnkretser = TRUE, 
#                                 Tid = as.character(aargang+1)) [[2]] %>%
#   dplyr::filter(!is.na(value)) %>%
#   dplyr::rename(GRUNNKRETSNUMMER = Grunnkretser,
#                 PERSONER = value) %>%
#   dplyr::select(GRUNNKRETSNUMMER, PERSONER)

# T04317 %>%
# dplyr::filter(GRUNNKRETSNUMMER %in% c("14440108", "14490114"))

In [7]:
# T04317_t1 <- PxWebApiData::ApiData(04317, ContentsCode = "Personer", 
#                                 Grunnkretser = TRUE, 
#                                 Tid = as.character(aargang)) [[2]] %>%
#   dplyr::filter(!is.na(value)) %>%
#   dplyr::rename(GRUNNKRETSNUMMER = Grunnkretser,
#                 PERSONER = value) %>%
#   dplyr::select(GRUNNKRETSNUMMER, PERSONER)

# T04317_t1 %>%
# dplyr::filter(GRUNNKRETSNUMMER %in% c("14440104", "14490113"))

## Kodeliste for opptaksområder i spesialisthelsetjenesten (somatikk)

In [8]:
opptaksomrader_KLASS <- klassR::GetKlass(629, output_style = "wide", date = c(paste0(aargang+1, "-01-01"))) %>%
dplyr::rename(GRUNNKRETSNUMMER = code4, 
             GRUNNKRETS_NAVN = name4, 
             OPPTAK_NUMMER = code3, 
             OPPTAK = name3, 
             ORGNR_HF = code2, 
             NAVN_HF = name2, 
             ORGNR_RHF = code1, 
             NAVN_RHF = name1)

nrow(opptaksomrader_KLASS)

[1] 14045

### Korrespondanse mellom t og t-1 fra KLASS

In [9]:
grunnkrets_KLASS <- klassR::GetKlass(1, date = c(paste0(aargang, "-01-01"), paste0(aargang+1, "-01-01")), correspond = TRUE) %>%
dplyr::rename(GRUNNKRETSNUMMER_T1 = sourceCode, 
             GRUNNKRETSNUMMER = targetCode, 
             targetName = targetName)

# grunnkrets_KLASS <- grunnkrets_KLASS %>%
# dplyr::filter(!GRUNNKRETSNUMMER_T1 %in% c("14440104") |  !GRUNNKRETSNUMMER %in% c("14490114")) # Fjerner Kjøs fra Markane

nrow(grunnkrets_KLASS)

[1] 241

OK

In [10]:
grunnkrets_KLASS %>%
group_by(GRUNNKRETSNUMMER) %>% # grunnkrets_KLASS_T1_2
filter(n()>1)

GRUNNKRETSNUMMER_T1,sourceName,GRUNNKRETSNUMMER,targetName
<chr>,<chr>,<chr>,<chr>
07200103,Råstad,07040118,Vearåsen
07200301,Stein,07040118,Vearåsen


In [11]:
grunnkrets_KLASS %>%
group_by(GRUNNKRETSNUMMER_T1) %>% # grunnkrets_KLASS_T1_2
filter(n()>1)

GRUNNKRETSNUMMER_T1,sourceName,GRUNNKRETSNUMMER,targetName
<chr>,<chr>,<chr>,<chr>
03010203,Sentrum 2 /rode 3,03010212,Sentrum 2/Rode 12
03010203,Sentrum 2 /rode 3,03010213,Sentrum 2/Rode 13
03011802,Sandaker rode 2,03011803,Sandaker Rode 3
03011802,Sandaker rode 2,03011804,Sandaker Rode 4
03011905,Åsen rode 5,03011909,Åsen Rode 9
03011905,Åsen rode 5,03011910,Åsen Rode 10
03012501,Grønland rode 1,03012507,Grønland rode 7
03012501,Grønland rode 1,03012508,Grønland rode 8
03012501,Grønland rode 1,03012509,Grønland rode 9


### Merger opptaksområder med KLASS (t-1)

In [12]:
opptaksomrader_KLASS_2 <- opptaksomrader_KLASS %>%
dplyr::left_join(grunnkrets_KLASS, by = "GRUNNKRETSNUMMER") %>%
dplyr::mutate(GRUNNKRETSNUMMER_T1 = case_when(
is.na(GRUNNKRETSNUMMER_T1) ~ GRUNNKRETSNUMMER, 
    TRUE ~ GRUNNKRETSNUMMER_T1
)) %>%
dplyr::select(GRUNNKRETSNUMMER, GRUNNKRETSNUMMER_T1, GRUNNKRETS_NAVN, OPPTAK_NUMMER, OPPTAK, ORGNR_HF, NAVN_HF, ORGNR_RHF, NAVN_RHF) %>%
dplyr::distinct(GRUNNKRETSNUMMER_T1, OPPTAK_NUMMER, OPPTAK, ORGNR_HF, NAVN_HF, ORGNR_RHF, NAVN_RHF)

nrow(opptaksomrader_KLASS_2)

[1] 14011

In [13]:
# opptaksomrader_KLASS_2 %>%
# dplyr::filter(GRUNNKRETSNUMMER_T1 == "18500109")

### Laster inn KLASS (t-1)

In [14]:
grunnkrets_KLASS_T1 <- klassR::GetKlass(1, output_style = "wide", date = c(paste0(aargang, "-01-01"))) %>%
dplyr::rename(GRUNNKRETSNUMMER_T1 = code2, 
             GRUNNKRETS_NAVN_T1 = name2) %>%
dplyr::select(GRUNNKRETSNUMMER_T1, GRUNNKRETS_NAVN_T1) 

nrow(grunnkrets_KLASS_T1)
nrow(opptaksomrader_KLASS_2)

[1] 14012

[1] 14011

In [15]:
# opptaksomrader_KLASS_2 %>%
# group_by(GRUNNKRETSNUMMER_T1) %>% # grunnkrets_KLASS_T1_2
# filter(n()>1)

In [16]:
nrow(grunnkrets_KLASS_T1)

grunnkrets_KLASS_T1_2 <- grunnkrets_KLASS_T1 %>%
dplyr::left_join(opptaksomrader_KLASS_2, by = "GRUNNKRETSNUMMER_T1") %>%
dplyr::filter(!is.na(OPPTAK))

nrow(grunnkrets_KLASS_T1_2)

# head(grunnkrets_KLASS_T1_2)

# grunnkrets_KLASS_T1_2 <- grunnkrets_KLASS_T1 %>%
# dplyr::distinct(GRUNNKRETSNUMMER_T1, GRUNNKRETS_NAVN_T1, OPPTAK)

[1] 14012

[1] 14011

In [17]:
grunnkrets_KLASS_T1_2 %>%
group_by(GRUNNKRETSNUMMER_T1) %>% # grunnkrets_KLASS_T1_2
filter(n()>1)

GRUNNKRETSNUMMER_T1,GRUNNKRETS_NAVN_T1,OPPTAK_NUMMER,OPPTAK,ORGNR_HF,NAVN_HF,ORGNR_RHF,NAVN_RHF
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>


In [18]:
grunnkrets_KLASS_T1_3 <- grunnkrets_KLASS_T1 %>%
dplyr::left_join(opptaksomrader_KLASS_2, by = "GRUNNKRETSNUMMER_T1") %>%
dplyr::filter(is.na(OPPTAK))

grunnkrets_KLASS_T1_3

GRUNNKRETSNUMMER_T1,GRUNNKRETS_NAVN_T1,OPPTAK_NUMMER,OPPTAK,ORGNR_HF,NAVN_HF,ORGNR_RHF,NAVN_RHF
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
99999999,Uoppgitt,NA,NA,NA,NA,NA,NA


## Laster inn tabell 04317

In [19]:
T04317 <- PxWebApiData::ApiData(04317, ContentsCode = "Personer", 
                                Grunnkretser = TRUE, 
                                Tid = as.character(aargang)) [[2]] %>%
  dplyr::filter(!is.na(value)) %>%
  dplyr::rename(GRUNNKRETSNUMMER = Grunnkretser,
                PERSONER = value) %>%
  dplyr::select(GRUNNKRETSNUMMER, PERSONER)

In [20]:
colnames(T04317)

[1] "GRUNNKRETSNUMMER" "PERSONER"

# OBS: Legg til opptaksområde for disse!

In [21]:
colnames(grunnkrets_KLASS_T1_2)

[1] "GRUNNKRETSNUMMER_T1" "GRUNNKRETS_NAVN_T1"  "OPPTAK_NUMMER"      
[4] "OPPTAK"              "ORGNR_HF"            "NAVN_HF"            
[7] "ORGNR_RHF"           "NAVN_RHF"

In [22]:
hovedopptak_per_kommune <- grunnkrets_KLASS_T1_2 %>%
# dplyr::filter(substr(GRUNNKRETSNUMMER_T1, 1, 4) == "0101") %>%
dplyr::mutate(KOMMUNENUMMER = substr(GRUNNKRETSNUMMER_T1, 1, 4)) %>%
dplyr::group_by(KOMMUNENUMMER, OPPTAK_NUMMER, OPPTAK, ORGNR_HF, NAVN_HF, ORGNR_RHF, NAVN_RHF) %>%
tally() %>%
dplyr::group_by(KOMMUNENUMMER) %>%
# dplyr::filter(n()>1)
dplyr::slice(which.max(n))

In [23]:
grunnkrets_KLASS_T1_4 <- grunnkrets_KLASS_T1_2 %>%
dplyr::full_join(T04317, by = c("GRUNNKRETSNUMMER_T1" = "GRUNNKRETSNUMMER")) %>%
dplyr::mutate(PERSONER = tidyr::replace_na(PERSONER, 0))

# grunnkrets_KLASS_T1_4 %>%
# dplyr::filter(is.na(OPPTAK), 
#              substr(GRUNNKRETSNUMMER_T1, 5, 8) != "9999")

grunnkrets_KLASS_T1_4 <- grunnkrets_KLASS_T1_4 %>%
dplyr::filter(is.na(OPPTAK)) %>%
dplyr::mutate(KOMMUNENUMMER = substr(GRUNNKRETSNUMMER_T1, 1, 4)) %>%
dplyr::select(GRUNNKRETSNUMMER_T1, KOMMUNENUMMER) %>%
dplyr::left_join(hovedopptak_per_kommune, by = c("KOMMUNENUMMER" = "KOMMUNENUMMER")) %>%
dplyr::select(-KOMMUNENUMMER, -n) %>%
dplyr::mutate(GRUNNKRETS_NAVN_T1 = "Uoppgitt grunnkrets")

# # Sjekker om antall personer stemmer med tabell 04317
# sum(T04317$PERSONER)-sum(grunnkrets_KLASS_T1_4$PERSONER)

In [24]:
head(grunnkrets_KLASS_T1_4)

,GRUNNKRETSNUMMER_T1,OPPTAK_NUMMER,OPPTAK,ORGNR_HF,NAVN_HF,ORGNR_RHF,NAVN_RHF,GRUNNKRETS_NAVN_T1
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,01019999,48,Østfold,983971768,SYKEHUSET ØSTFOLD HF,991324968,HELSE SØR-ØST RHF,Uoppgitt grunnkrets
2,01049999,48,Østfold,983971768,SYKEHUSET ØSTFOLD HF,991324968,HELSE SØR-ØST RHF,Uoppgitt grunnkrets
3,01059999,48,Østfold,983971768,SYKEHUSET ØSTFOLD HF,991324968,HELSE SØR-ØST RHF,Uoppgitt grunnkrets
4,01069999,48,Østfold,983971768,SYKEHUSET ØSTFOLD HF,991324968,HELSE SØR-ØST RHF,Uoppgitt grunnkrets
5,01119999,48,Østfold,983971768,SYKEHUSET ØSTFOLD HF,991324968,HELSE SØR-ØST RHF,Uoppgitt grunnkrets
6,01199999,48,Østfold,983971768,SYKEHUSET ØSTFOLD HF,991324968,HELSE SØR-ØST RHF,Uoppgitt grunnkrets


In [25]:
head(grunnkrets_KLASS_T1_2)

,GRUNNKRETSNUMMER_T1,GRUNNKRETS_NAVN_T1,OPPTAK_NUMMER,OPPTAK,ORGNR_HF,NAVN_HF,ORGNR_RHF,NAVN_RHF
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,01010102,Båstadlund,48,Østfold,983971768,SYKEHUSET ØSTFOLD HF,991324968,HELSE SØR-ØST RHF
2,01010103,Låby,48,Østfold,983971768,SYKEHUSET ØSTFOLD HF,991324968,HELSE SØR-ØST RHF
3,01010104,Stangeløkka,48,Østfold,983971768,SYKEHUSET ØSTFOLD HF,991324968,HELSE SØR-ØST RHF
4,01010105,Refne/Banken,48,Østfold,983971768,SYKEHUSET ØSTFOLD HF,991324968,HELSE SØR-ØST RHF
5,01010107,Remmen,48,Østfold,983971768,SYKEHUSET ØSTFOLD HF,991324968,HELSE SØR-ØST RHF
6,01010108,Refne,48,Østfold,983971768,SYKEHUSET ØSTFOLD HF,991324968,HELSE SØR-ØST RHF


In [26]:
grunnkrets_KLASS_T1_2 <- rbind(grunnkrets_KLASS_T1_2, grunnkrets_KLASS_T1_4)

In [27]:
grunnkrets_KLASS_T1_5 <- grunnkrets_KLASS_T1_2 %>%
dplyr::full_join(T04317, by = c("GRUNNKRETSNUMMER_T1" = "GRUNNKRETSNUMMER")) %>%
dplyr::mutate(PERSONER = tidyr::replace_na(PERSONER, 0))

sum(grunnkrets_KLASS_T1_5$PERSONER)-sum(T04317$PERSONER)

[1] 0

### Lagrer filen 

In [31]:
test <- grunnkrets_KLASS_T1_2 %>%
  dplyr::mutate(TOM_FORELDER = "") %>%
  dplyr::select(GRUNNKRETSNUMMER_T1, GRUNNKRETS_NAVN_T1, OPPTAK_NUMMER, ORGNR_HF, NAVN_HF, OPPTAK, ORGNR_RHF, NAVN_RHF, TOM_FORELDER)

level_1 <- test %>%
  dplyr::select(ORGNR_RHF, TOM_FORELDER, NAVN_RHF) %>%
  dplyr::rename('ns1:kode' = ORGNR_RHF, 
                'ns1:forelder' = TOM_FORELDER, 
                'ns1:navn_bokmål' = NAVN_RHF) %>%
  dplyr::distinct()

level_2 <- test %>%
  dplyr::select(ORGNR_HF, ORGNR_RHF, NAVN_HF) %>%
  dplyr::rename('ns1:kode' = ORGNR_HF, 
                'ns1:forelder' = ORGNR_RHF, 
                'ns1:navn_bokmål' = NAVN_HF) %>%
  dplyr::distinct()

level_3 <- test %>%
  dplyr::select(OPPTAK_NUMMER, ORGNR_HF, OPPTAK) %>%
  dplyr::rename('ns1:kode' = OPPTAK_NUMMER, 
                'ns1:forelder' = ORGNR_HF, 
                'ns1:navn_bokmål' = OPPTAK) %>%
  dplyr::distinct()

level_4 <- test %>%
  dplyr::select(GRUNNKRETSNUMMER_T1, OPPTAK_NUMMER, GRUNNKRETS_NAVN_T1) %>%
  dplyr::rename('ns1:kode' = GRUNNKRETSNUMMER_T1, 
                'ns1:forelder' = OPPTAK_NUMMER, 
                'ns1:navn_bokmål' = GRUNNKRETS_NAVN_T1) %>%
  dplyr::distinct()

KLASS <- rbind(level_1, level_2, level_3, level_4) %>%
dplyr::select('ns1:kode', 'ns1:navn_bokmål', 'ns1:forelder')

In [33]:
openxlsx::write.xlsx(KLASS, file = paste0("/ssb/bruker/rdn/opptak_", aargang, ".xlsx"),
                     rowNames = FALSE,
                     showNA = FALSE,
                     overwrite=T) # T = overskriver dersom filen allerede finnes, F = gir feilmelding dersom filen finnes